In [3]:
import json
import os
import re
import sys

import pandas as pd
import numpy as np

sys.path.append(os.path.dirname(os.path.abspath('')))

from app.utils import translit
from app.settings import TRANSPORT_TYPE_COLORS, TRANSPORT_TYPE_NAMES

In [4]:
df1 = pd.read_excel("./monday.xlsx")
df2 = pd.read_excel("./saturday.xlsx")
df3 = pd.read_excel("./sunday.xlsx")

data_columns = []
columns = {
    df1.columns[0]: "name" 
}
for i in range(1, len(df1.columns)):
    new_name = f"data{i}"
    columns[df1.columns[i]] = new_name
    data_columns.append(new_name)
    
df1.rename(columns=columns, inplace=True)
df2.rename(columns=columns, inplace=True)
df3.rename(columns=columns, inplace=True)
df1.head(10)

,name,data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,В будничные дни,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,МАРШРУТЫ,ВРЕМЯ ОТПРАВЛЕНИЯ С АВТОВОКЗАЛА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,№2 Автовокзал - Старокамышинск,5:15,5:45,6:00,6:15,6:38,7:11,7:26,7:41,8:04,8:37,9:05,9:32
5,NaN,10:25,10:36,10:58,11:13,11:51,12:22,12:39,13:15,14:08,14:30,14:56,15:34
6,NaN,15:56,16:10,16:22,17:04,17:36,17:48,18:02,19:02,19:15,19:33,20:04,20:38
7,NaN,21:03,21:30,22:00,23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,№4 Автовокзал - ДК Петяркова,6:05,6:55,7:45,8:40,9:33,11:25,12:15,13:05,13:55,14:55,15:45,16:35
9,NaN,17:28,19:05,19:55,20:45,21:35,22:25,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
routes = {}

name = ''
times = []

def update_route_data(routes, name, times_name, times):
    if not times:
        return
    
    key = re.sub(r'[^\w]', '', str(name).lower())
    
    if times_name != 'monday' and key not in routes:
        raise Exception(f'Unknown route {name}')
    
    if key not in routes:
        routes[key] = {
            'name': name,
            'monday': [],
            'saturday': [],
            'sunday': [],
        }
    routes[key][times_name] = times

for df, times_name in (
    (df1, 'monday',),
    (df2, 'saturday',),
    (df3, 'sunday',),
):
    df = df.replace({np.nan: None})
    for index, data in df.iterrows():
        if not data['data1']:
            continue
        if 'время' in str(data['data1']).lower():
            continue
        if data['name']:
            update_route_data(routes, name, times_name, times)
            name = data['name']
            times = []
        for i in data_columns:
            if data[i]:
                times.append(data[i])
    update_route_data(routes, name, 'monday', times)
    
def get_json_data(route):
    if route['monday'] == route['saturday']:
        return (
            (route['name'], ('Будни и Суббота', 'Воскресенье и праздники'), (route['monday'], route['sunday'])),
        )
    elif route['saturday'] == route['sunday']:
        return (
            (route['name'], ('Будни', 'Выходные и праздники'), (route['monday'], route['sunday'])),
        )
    else:
        return (
            (route['name'], ('Будни', ''), (route['monday'], [])),
            (route['name'], ('Суббота', ''), (route['saturday'], [])),
            (route['name'], ('Воскресенье и праздники', ''), (route['sunday'], [])),
        )
    
def get_json_payload(name, titles_list, times_list):
    rows = []
    for times in times_list:
        current_dict = {}
        for item in times:
            hour, minute = item.split(":")
            hour = hour.zfill(2)
            minute = minute.zfill(2)
            if hour not in current_dict:
                current_dict[hour] = []
            current_dict[hour].append(minute)
        for _, v in current_dict.items():
            if "" not in v:
                v.append("")
        rows.append(
            {
                "timesByHour": current_dict,
            }
        )
    
    payload = {
        "head": {
            "type": TRANSPORT_TYPE_NAMES["bus"],
            "routeNumber": name.split(' ')[0],
            "color": TRANSPORT_TYPE_COLORS["bus"],
            "direction": name,
            "description": ".",
            "shiftMinutes": 0,
            "pdfFilename": translit(name),
        },
        "body": {
            "columnHeads": titles_list,
            "rows": [
                {
                    "name": "Рейсы",
                    "columns": rows,
                }
            ],
        },
        "commentBottom": [],
    }
        
    return payload
    
for route in routes.values():
    for data in get_json_data(route):
        payload = get_json_payload(*data)
        payload = json.dumps(payload, indent=4, ensure_ascii=False)
        print('---------------------------------------------------------------------------')
        print(data[0])
        print('---------------------------------------------------------------------------')
        print(str(payload))
        print('\n\n')

---------------------------------------------------------------------------
№2 Автовокзал - Старокамышинск
---------------------------------------------------------------------------
{
    "head": {
        "type": "Автобус",
        "routeNumber": "№2",
        "color": [
            85,
            22,
            100,
            13
        ],
        "direction": "№2 Автовокзал - Старокамышинск",
        "description": ".",
        "shiftMinutes": 0,
        "pdfFilename": "№2_Avtovokzal_-_Starokamysinsk"
    },
    "body": {
        "columnHeads": [
            "Будни",
            "Выходные и праздники"
        ],
        "rows": [
            {
                "name": "Рейсы",
                "columns": [
                    {
                        "timesByHour": {
                            "05": [
                                "15",
                                "45",
                                ""
                            ],
                            "06": [
 